In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Reading data

In [ ]:
# Reading the filtered and final crash files for each year

crash_2016 = pd.read_csv('data/crash/crash_2016.csv')
crash_2017 = pd.read_csv('data/crash/crash_2017.csv')
crash_2018 = pd.read_csv('data/crash/crash_2018.csv')

crash_list = [crash_2016, crash_2017, crash_2018]
crash = pd.concat(crash_list)

crash.loc[crash.trav_sp==99,'trav_sp'] = 999
crash.loc[crash.trav_sp==98,'trav_sp'] = 999

crash = crash.drop_duplicates().reset_index(drop=True)

In [ ]:
pd.set_option('display.max_columns', 200) # to display all 51 columns 
crash.head()

In [ ]:
plt.plot(crash.mdlyr_im.value_counts()[crash.mdlyr_im.value_counts().index.isin(range(1995,2020))].sort_index().index, 
         crash.mdlyr_im.value_counts()[crash.mdlyr_im.value_counts().index.isin(range(1995,2020))].sort_index())

#### Finding 1:

The number of 2008-2011 year vehicles are lower than expected. It must be related with financial crisis of 2007-2008 and its long-term effects

## Damage to people / first level

#### Quantizing the severity of damage to people in a vehicle

Here, I need to create a column which will show the severity of damage from 100 (fatal) to 0 (no injury)

Assumption: Injured, Severity Unknown must be in between "Minor Injury" (50) and "Serious Injury" (75). Hence it is quantized as 65

In [ ]:
# The numbers of each injury severity levels
crash.inj_sev_rate.value_counts()

#### Grouping by make_models

In [ ]:
avg_inj_sev = crash[['make_model', 'inj_sev_rate', 'vinyear']].groupby('make_model').agg(['mean', 'count'])
avg_inj_sev.columns = ['avg_inj_sev', 'number_of_accidents', 'avg_mod_year', 'count']
avg_inj_sev = avg_inj_sev.drop(columns='count')
avg_inj_sev.avg_mod_year = round(avg_inj_sev.avg_mod_year, 1)
avg_inj_sev = avg_inj_sev.sort_values('avg_inj_sev',ascending=True)
avg_inj_sev = avg_inj_sev.reset_index()

#### Question:

What should be the minimum sample number to be able to speak statistically a make/model is safe or not?

In [ ]:
# Sample number of specific make/model
n = 50
avg_inj_sev[avg_inj_sev.number_of_accidents>n].head()

#### Problem:

This maximum severity of injury shows only the severity of the most injured person in a vehicle.

Another important factor is what percentage of people in vehicle got injured?

#### Possible solution:

Creating weighted severity of injury: inury severity * number of injured / number of occupants  

In [ ]:
# weighted_inj_sev_rate = inj_sev_rate*numinj/numoccs
# This is to make weighting process smoother, if inj_sev=100, I do not want to decrease it to 0, it`ll decrease the number 1/2*inj_sev at most
crash.weighted_inj_sev_rate = (crash.inj_sev_rate+crash.weighted_inj_sev_rate)/2


In [ ]:
avg_wgh_inj_sev = crash[['make_model', 'weighted_inj_sev_rate', 'vinyear']].groupby('make_model').agg(['mean', 'count'])
avg_wgh_inj_sev.columns = ['avg_wgh_inj_sev', 'number_of_accidents', 'avg_mod_year', 'count']
avg_wgh_inj_sev = avg_wgh_inj_sev.drop(columns='count')
avg_wgh_inj_sev.avg_mod_year = round(avg_wgh_inj_sev.avg_mod_year, 1)
avg_wgh_inj_sev = avg_wgh_inj_sev.sort_values('avg_wgh_inj_sev',ascending=True)
avg_wgh_inj_sev = avg_wgh_inj_sev.reset_index()

In [ ]:
avg_wgh_inj_sev[avg_wgh_inj_sev.number_of_accidents>n].head()

I need to go deeper since it is obvious that not all the accident conditions are exact same. I should compare similiar conditions and results in order to see more accurate. Hence I need to create another level which will show the condition in which crash occured.

## Crash condition / second level

In [ ]:
# Creating new column to display and categorize travel speeds as multiples of 5 i.e. [0-5)-> 0, [5-10)-> 5, [10-15)-> 10, ... mph 

# speed_interval = pd.interval_range(start=0, freq=5, end=150, closed='left')
# speed_list=[]
# for i in range(0,150,5): speed_list.append(i)
# crash['travel_speed'] = np.where(crash.trav_sp==997, 
#                                                        160,
#                                                        pd.cut(crash.trav_sp, bins=speed_interval).apply(lambda x: x.left)
#                                                       )
# crash.travel_speed = crash.travel_speed.fillna(999).astype(int)

# Doing the same work:

crash['travel_speed'] = (crash.trav_sp/5).astype(int)*5

crash.loc[(crash.travel_speed>=90) & (crash.travel_speed<100),'travel_speed'] = 90

crash.loc[(crash.travel_speed>=100) & (crash.travel_speed<160),'travel_speed'] = 100

In [ ]:
# The number of vehicles older than 1989 is lower than 100, so I want to group them to see if there is a correlation between year and damage
# Creating new column to display model years of old cars as group:
# [:,1970)-> 1960, [1970,1980)-> 1970, [1980-1990)->1980

crash['year'] = crash.mdlyr_im.astype(int)

crash.loc[(crash.year<1990), 'year'] = (crash.mdlyr_im/10).astype(int)*10

# crash['year'] = np.where(crash.mdlyr_im<1990,
#                                                (crash.mdlyr_im/10).astype(int)*10,
#                                                crash.mdlyr_im.astype(int) )

** The conditions that may have huge impact on the damage resulted. I want to see their effects and then isolate that effect if I could.**

In [ ]:
condition_list = ['travel_speed', 
                  'weathr_im', 'lgtcon_im',
                  'mfactor', 
                  'vevent_im', 'impact1_im',
                  'year',
                  'vsurcond', 
                  'v_alch_im' ]

crush_type_list = ['acc_type', 'rollover', 'deformed', 'fire_exp']

I will create a dataframe which contains all groups of all these categories. Hence I first created a function which group by by a category and returns the counts and avgerage severity of injury for each groups(which is code for each category/data elemnts). Then I merged these data for all conditions.

In [ ]:
# This is the average of all conditions in 2016
overall_avg = crash.weighted_inj_sev_rate.describe()[['mean', 'min', '25%', '50%', '75%', 'max']].mean()

# Minimum number of crushes in order to be able to safely say that a condition is more dangerous or safe 
min_cnt = 25

def category_avg(category, rank=0):
    if rank==0:
        damage='weighted_inj_sev_rate'
    else:
        damage='damage_over_condition'
    overall_avg = crash[damage].describe()[['mean', 'min', '25%', '50%', '75%', 'max']].mean()
    condition = crash[[category, damage]].groupby(category).describe()
    condition.columns = [category+'_count', 'mean', 'std', 'min', '1q', '2q', '3q', 'max']
    condition[category+'_avg_severity'] = condition[['mean', 'min', '1q', '2q', '3q', 'max']].mean(axis=1)
    condition[category+'_relative_severity'] = (condition[category+'_avg_severity']-overall_avg).round(2)
    condition = condition[condition[category+'_count']>min_cnt].reset_index()
    condition = condition.drop(columns=[category+'_count',category+'_avg_severity', 'mean', 'std', 'min', '1q', '2q', '3q', 'max']).reset_index(drop=True)
    
    return condition

conditions = pd.DataFrame()

for i in range(len(condition_list+crush_type_list)):
    
    conditions = pd.merge(conditions, category_avg((condition_list+crush_type_list)[i]), left_index=True, right_index=True, how='outer')
    
conditions.head()

In [ ]:
# category_avg('travel_speed',1)

This 'conditions' dataframe will show us the relative severity for each condition

In [ ]:
crash['crash_condition'] = 0

crash['damage_over_condition'] = crash.weighted_inj_sev_rate - crash.crash_condition

for i in range(len(condition_list)):
    
    if i==0:
        damage='weighted_inj_sev_rate'
    else:
        damage='damage_over_condition'
    
    crash = pd.merge(crash, category_avg(condition_list[i],i), on= condition_list[i], how='left')
        
    crash[condition_list[i]+'_relative_severity'] = crash[condition_list[i]+'_relative_severity'].fillna(0)
    
    crash.loc[crash[condition_list[i]+'_relative_severity']<-10, condition_list[i]+'_relative_severity'] = -8 + (1/5)*crash[condition_list[i]+'_relative_severity']
    crash.loc[((crash[condition_list[i]+'_relative_severity']>-10) & (crash[condition_list[i]+'_relative_severity']<-5)), condition_list[i]+'_relative_severity'] = -5 + (3/5)*crash[condition_list[i]+'_relative_severity']
    crash.loc[((crash[condition_list[i]+'_relative_severity']<10) & (crash[condition_list[i]+'_relative_severity']>5)), condition_list[i]+'_relative_severity'] = 5 + (3/5)*crash[condition_list[i]+'_relative_severity']
    crash.loc[crash[condition_list[i]+'_relative_severity']>10, condition_list[i]+'_relative_severity'] = 8 + (1/5)*crash[condition_list[i]+'_relative_severity']
    
    crash.crash_condition = crash.crash_condition + crash[condition_list[i]+'_relative_severity']
    
    crash.loc[(((crash[damage]<=0) & (crash[condition_list[i]+'_relative_severity'].fillna(0)>0)) | (crash[damage]!=0)),'damage_over_condition'] = crash.damage_over_condition - 1.5*crash[condition_list[i]+'_relative_severity'].fillna(0)
        
    crash = crash.drop(columns=condition_list[i]+'_relative_severity')

crash.damage_over_condition = 100*(crash.damage_over_condition-crash.damage_over_condition.min())/(crash.damage_over_condition.max()-crash.damage_over_condition.min())

crash['safety_rate'] = 100-crash.damage_over_condition

crash.head()

In [ ]:
# from scipy import stats

# z = np.abs(stats.zscore(crash.safety_rate))

# crash[z>2].safety_rate.hist()

In [ ]:
avg_safety_rate = crash[['make_model', 'safety_rate', 'vinyear', 'weighted_inj_sev_rate']].groupby('make_model').agg(['mean', 'count'])
avg_safety_rate.columns = ['avg_safety_rate', 'number_of_accidents', 'avg_mod_year', 'count', 'weighted_inj_sev_rate', 'cnt']
avg_safety_rate = avg_safety_rate.drop(columns=['count', 'cnt'])
avg_safety_rate.avg_mod_year = round(avg_safety_rate.avg_mod_year, 1)
avg_safety_rate = avg_safety_rate.sort_values('avg_safety_rate',ascending=True)
avg_safety_rate = avg_safety_rate.reset_index()
avg_safety_rate[avg_safety_rate.number_of_accidents>2].tail()

** It is time to get 2017 and 2018 crash data and merge into a final crash file. I will use the same metrics created in this notebook for combined data for three years(2016, 2017, 2018). **

In [ ]:
crash = crash[~crash.make.isin(['DAEWOO', 'EAGLE', 'DATSUN', 'KENWORTH','FRUEHAUF','KAR-TOTE','THE VEHICLE PRODUCTION GROUP','FREIGHTLINER','WORKHORSE','AM GENERAL','CLASSIC MOTORCYCLES & SIDECARS','AMERICAN MOTORS','WORKHORSE CUSTOM CHASSIS','WILSON TRAILER CO'])].reset_index(drop=True)

crash = crash[~crash.bdytyp_im.isin([11,41,12])].reset_index(drop=True)

crash.loc[crash.make=='ROLLS-ROYCE','make'] = 'ROLLS ROYCE'

crash.loc[crash.make=='RAM','model'] = 'RAM ' + crash.loc[crash.make=='RAM','model'].astype(str)

crash.loc[crash.make=='RAM','make'] = 'DODGE'

crash.make_model = crash.make + ' ' + crash.model

In [ ]:
body_codes = [4, 14, 34, 6, 15, 2, 20, 31, 5, 3, 30, 1, 16, 9, 40, 8, 17, 39, 10, 19, 48, 12, 45, 7, 32, 42]

body_names = ['Sedan', 'SUV', 'Pickup', 'Station Wagon', 'SUV', 'Sedan 2-Door/       Coupe',
             'Minivan', 'Pickup', 'Hatchback', 'Hatchback/  2-Door',
              'Pickup', 'Convertible', 'Station Wagon', 'Sedan', 
             'Pickup', 'Sedan', 'Sedan 2-Door/       Coupe', 'Pickup', 'Pickup', 'SUV', 'Pickup',
             'Limousine', 'Pickup', 'Hatchback', 'Pickup','Pickup']

body_decoding = pd.DataFrame(index=body_codes, columns=['body_type'],data=body_names)
body_decoding = body_decoding.reset_index().rename(columns={'index':'bdytyp_im'})

crash = pd.merge(crash,body_decoding, on='bdytyp_im', how='left')

## Adding car price data

In [ ]:
price = pd.read_csv('data/msrp_data.csv')
price.columns = price.columns.str.lower()
price.make = price.make.str.upper()
price.model = price.model.str.upper()
price.loc[price.make=='ROLLS-ROYCE','make'] = 'ROLLS ROYCE'
price['make_model'] = price.make + ' ' + price.model
price.head()

In [ ]:
price_grouped = price[['make_model', 'year', 'msrp', 'highway mpg', 'city mpg']][(price.make_model.isin(crash.make_model)) & (price.year>2000)].groupby(by=['make_model', 'year']).mean().astype(int).reset_index()
price_grouped = price_grouped.groupby('make_model').last().reset_index().rename(columns={'year':'msrp_year'})
price_grouped.columns = ['make_model', 'msrp_year', 'msrp', 'highway_mpg', 'city_mpg']
price_grouped.head()

In [ ]:
crash = pd.merge(crash, price_grouped, on='make_model',how='left')
crash.head()

In [ ]:
print('The ' + str(round(100*(crash.shape[0]-crash.msrp.isna().sum())/crash.shape[0],2)) + ' percent of the vehicles in my crash data are matched with the price data in terms of make-model')

print('The ' + str(round(100*crash[~crash.msrp.isna()].make_model.unique().shape[0]/crash.make_model.unique().shape[0],2)) + ' percent of the Make-Model pairs in my crash data are matched with the price data')

In [ ]:
# limit = -3
def zero_inj_rate(limit):
    return 100*crash[((crash.crash_condition<=limit)&(crash.inj_sev_rate==0))].shape[0]/crash[((crash.crash_condition<=limit))].shape[0]
zero_rate = []
limit_range = []
for i in range(-10,10):
    zero_rate.append(zero_inj_rate(i))
    limit_range.append(i)
plt.plot(limit_range,zero_rate)

In [ ]:
# If the crash condition severity is well below the average and there is no injury, it does not show that the vehicle is safe! So I want to exlude that case.
crash3 = crash[~((crash.crash_condition<=-3)&(crash.inj_sev_rate==0))].reset_index(drop=True)

In [ ]:
crash.to_csv('data/crash2.csv', index=False)
crash3.to_csv('data/crash3.csv', index=False)